In [ ]:
import os, sys

# Get the project root directory (one level above 'notebooks')
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to Python path if not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now you can safely import from app/
from app.config import settings

print("✅ Import successful! Database URL:", settings.database_url)

In [ ]:
# ─────────────────────────────────────────────
# 1️⃣  Setup – connect to Postgres
# ─────────────────────────────────────────────
import pandas as pd
from sqlalchemy import create_engine
from app.config import settings  # uses your .env automatically

engine = create_engine(settings.database_url)
print("Connected to:", settings.database_url)

In [ ]:
# ─────────────────────────────────────────────
# 2️⃣  Load all messages
# ─────────────────────────────────────────────
query = """
SELECT
    id,
    channel_id,
    user_id,
    text,
    ts::double precision AS ts_float,
    to_timestamp(ts::double precision) AS message_time,
    thread_ts,
    edited_ts,
    deleted
FROM messages
ORDER BY ts_float;
"""
messages = pd.read_sql(query, engine)
messages.head()

In [ ]:
# ─────────────────────────────────────────────
# 3️⃣  Thread overview (parent → replies)
# ─────────────────────────────────────────────
threads = pd.read_sql("""
    SELECT
        parent.text AS parent_message,
        child.text  AS reply_message,
        to_timestamp(child.ts::double precision) AS reply_time,
        child.user_id
    FROM messages child
    LEFT JOIN messages parent
        ON child.thread_ts = parent.ts
    ORDER BY parent.ts::numeric NULLS LAST, child.ts::numeric;
""", engine)

threads.head(10)

In [ ]:
# ─────────────────────────────────────────────
# 4️⃣  Message activity over time
# ─────────────────────────────────────────────
import matplotlib.pyplot as plt

messages["date"] = messages["message_time"].dt.date
daily_counts = messages.groupby("date").size()

plt.figure(figsize=(8,4))
plt.plot(daily_counts.index, daily_counts.values, marker="o")
plt.title("Daily Slack Message Activity")
plt.xlabel("Date")
plt.ylabel("Messages")
plt.grid(True)
plt.show()

In [ ]:
# ─────────────────────────────────────────────
# 5️⃣  Top users by message count
# ─────────────────────────────────────────────
top_users = (
    messages.groupby("user_id")
    .size()
    .sort_values(ascending=False)
    .reset_index(name="message_count")
)
top_users.head(10)


In [ ]:
# ─────────────────────────────────────────────
# 🔚 Safe shutdown cell
# ─────────────────────────────────────────────
try:
    engine.dispose()
    print("✅ Database connections closed.")
except NameError:
    print("ℹ️ No engine defined (nothing to close).")

# Optional: clear large DataFrames to free memory
for name in list(globals()):
    if isinstance(globals()[name], pd.DataFrame):
        del globals()[name]
print("🧹 Memory cleaned up. You can now safely stop the kernel.")

In [ ]:
import IPython
IPython.get_ipython().kernel.do_shutdown(restart=False)